In [1]:
import os
import angr, claripy


filePath = '/home/nahid/reverse/binaries/pass'
proj = angr.Project(filePath, auto_load_libs=False)

WARNING  | 2023-02-05 17:49:17,322 | cle.loader     | The main binary is a position-independent executable. It is being loaded with a base address of 0x400000.


In [2]:


state = proj.factory.entry_state()
simgr = proj.factory.simgr(state)
simgr.active


simgr.step()
simgr.active


[<SimState @ 0x500000>]

In [3]:
# Step until the first symbolic branch
while len(simgr.active) == 1:
    simgr.step()

print(simgr)
# <SimulationManager with 2 active>
print(simgr.active)
# [<SimState @ 0x400692>, <SimState @ 0x400699>]

# Step until everything terminates
simgr.run()
print(simgr)
# <SimulationManager with 3 deadended>

WARNING  | 2023-02-05 17:50:17,415 | angr.storage.memory_mixins.default_filler_mixin | The program is accessing memory with an unspecified value. This could indicate unwanted behavior.
WARNING  | 2023-02-05 17:50:17,416 | angr.storage.memory_mixins.default_filler_mixin | angr will cope with this by generating an unconstrained symbolic variable and continuing. You can resolve this by:
WARNING  | 2023-02-05 17:50:17,417 | angr.storage.memory_mixins.default_filler_mixin | 1) setting a value to the initial state
WARNING  | 2023-02-05 17:50:17,417 | angr.storage.memory_mixins.default_filler_mixin | 2) adding the state option ZERO_FILL_UNCONSTRAINED_{MEMORY,REGISTERS}, to make unknown regions hold null
WARNING  | 2023-02-05 17:50:17,417 | angr.storage.memory_mixins.default_filler_mixin | 3) adding the state option SYMBOL_FILL_UNCONSTRAINED_{MEMORY,REGISTERS}, to suppress these messages.
WARNING  | 2023-02-05 17:50:17,418 | angr.storage.memory_mixins.default_filler_mixin | Filling memory at 0

<SimulationManager with 2 active>
[<SimState @ 0x40124a>, <SimState @ 0x401251>]
<SimulationManager with 3 deadended>


In [4]:
simgr.move(from_stash='deadended', to_stash='authenticated', filter_func=lambda s: b'Welcome' in s.posix.dumps(1))
print(simgr)

<SimulationManager with 1 deadended, 2 authenticated>


In [5]:
for s in simgr.deadended + simgr.authenticated:
    print(hex(s.addr))


print(simgr.one_deadended)

print(simgr.mp_authenticated)

print(simgr.mp_authenticated.posix.dumps(0))


0x500038
0x601050
0x601050
<SimState @ 0x500038>
MP([<SimState @ 0x601050>, <SimState @ 0x601050>])
MP([b'\x00\x00\x00\x00\x00\x00\x00\x00\x00SOSNEAKY\x00', b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x10 \x10@\x00\x00\x00'])
